In [1]:
import requests
import pandas as pd
from fetch_data import OSMR_Client

# read data to dataframe
df = pd.read_csv('raw_data.csv', sep=',', decimal='.')
display(df.head(10))
print(f"No of raw records: {len(df)}")


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30
5,0x9bb4,HYDRES09DEL03,22,4.8,17.431668,78.408321,17.461668,78.438321,11-03-2022,21:20:00,21:30:00,conditions Cloudy,Jam,0,Buffet,motorcycle,1,No,Urban,(min) 26
6,0x95b4,RANCHIRES15DEL01,33,4.7,23.369746,85.339820,23.479746,85.449820,04-03-2022,19:15:00,19:30:00,conditions Fog,Jam,1,Meal,scooter,1,No,Metropolitian,(min) 40
7,0x9eb2,MYSRES15DEL02,35,4.6,12.352058,76.606650,12.482058,76.736650,14-03-2022,17:25:00,17:30:00,conditions Cloudy,Medium,2,Meal,motorcycle,1,No,Metropolitian,(min) 32
8,0x1102,HYDRES05DEL02,22,4.8,17.433809,78.386744,17.563809,78.516744,20-03-2022,20:55:00,21:05:00,conditions Stormy,Jam,0,Buffet,motorcycle,1,No,Metropolitian,(min) 34
9,0xcdcd,DEHRES17DEL01,36,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55:00,22:10:00,conditions Fog,Jam,2,Snack,motorcycle,3,No,Metropolitian,(min) 46


No of raw records: 45593


In [2]:
# bounding box of India coordinates
india_bbox = (68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078)
india_high_lat, india_low_lat = 35.4940095078, 7.96553477623
india_high_long, india_low_long = 97.4025614766, 68.1766451354

# dropping rows with NaN (although there is no NaNs)
df.dropna(inplace=True)

# dropping locations outside of bbox
outside_locations_df = df.loc[(df['Restaurant_latitude'] < india_low_lat) |
                              (df['Restaurant_latitude'] > india_high_lat) |
                              (df['Restaurant_longitude'] < india_low_long) |
                              (df['Restaurant_longitude'] > india_high_long) |
                              (df['Delivery_location_latitude'] < india_low_lat) |
                              (df['Delivery_location_latitude'] > india_high_lat) |
                              (df['Delivery_location_longitude'] < india_low_long) |
                              (df['Delivery_location_longitude'] > india_high_long)]

df.drop(outside_locations_df.index, inplace=True)



In [8]:
client = OSMR_Client()
i = 0
for id, x1,y1,x2,y2 in zip(df['ID'],df['Restaurant_longitude'], df['Restaurant_latitude'], df['Delivery_location_longitude'], df['Delivery_location_latitude']):
    client.get_route_json(id,start_point=(x1,y1), end_point=(x2,y2))
    print(i)
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183


KeyboardInterrupt: 

In [5]:
from routingpy import OSRM

In [6]:
# collecting road distances
client = OSRM()

#PLEASE RUN CAREFULLY PLEASE

# x1,y1,x2,y2 = df['Restaurant_longitude'][0], df['Restaurant_latitude'][0], df['Delivery_location_longitude'][0], df['Delivery_location_latitude'][0]

iter = 1
route_distances = []
exception_cases = []
for x1,y1,x2,y2 in zip(df['Restaurant_longitude'], df['Restaurant_latitude'], df['Delivery_location_longitude'], df['Delivery_location_latitude']):
    try:
        coords = [[x1,y1], [x2,y2]]
        route = client.directions(locations=coords)
        route_distances.append(route.distance/1000)
    except Exception as e:
        print(e, f"\tIter num: {iter}")
        exception_cases.append(iter)
    iter += 1
    print(iter)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
400 ({"message":"Impossible route between points","code":"NoRoute"}) 	Iter num: 24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


KeyboardInterrupt: 

In [ ]:
# write exception cases to file to not loose them
print(exception_cases)

[23, 376, 479, 561, 765, 1018, 1021, 1154, 1407, 1458, 1507, 1566, 1872, 1934, 1991, 2106, 2277, 2419, 2455, 2913, 2935, 2988, 3066, 3084, 3125, 3204, 3229, 3403, 3530, 3693, 3704, 3750, 3874, 3964, 4065, 4459, 4615, 4965, 5036, 5193, 5202, 5244, 5305, 5716, 6118, 6378, 6618, 6755, 6996, 7144, 7242, 7501, 7599, 8174, 8298, 8514, 9122, 9165, 9370, 9397, 9693, 9805, 10043, 10352, 10396, 10519, 10583, 10808, 10828, 11030, 11257, 11286, 11325, 11344, 11621, 11641, 11643, 11713, 11742, 11838, 12124, 12198, 12223, 12350, 12361, 12706, 13014, 13320, 13419, 13508, 13512, 13648, 13869, 14148, 14171, 14428, 14578, 14688, 14897, 14924, 14927, 15420, 15690, 15898, 15909, 16129, 16210, 16323, 16375, 16394, 16404, 16505, 16845, 16909, 16980, 17088, 17444, 17508, 18364, 18684, 19557, 19586, 19661, 19765, 20003, 20276, 20318, 20321, 20688, 20742, 20879, 21015, 21227, 21271, 21698, 21732, 21826, 21934, 21947, 21991, 22111, 22159, 22236, 22400, 22455, 22833, 22910, 22968, 23002, 23063, 23176, 23244, 233

In [ ]:
# write good routes to file to not loose them
with open('good_routes_distances.txt', 'w') as f:
    for line in route_distances:
        f.write(f"{line}\n")

In [ ]:
exception_cases_match_df = [x-1 for x in exception_cases]
exception_cases_match_df

[22,
 375,
 478,
 560,
 764,
 1017,
 1020,
 1153,
 1406,
 1457,
 1506,
 1565,
 1871,
 1933,
 1990,
 2105,
 2276,
 2418,
 2454,
 2912,
 2934,
 2987,
 3065,
 3083,
 3124,
 3203,
 3228,
 3402,
 3529,
 3692,
 3703,
 3749,
 3873,
 3963,
 4064,
 4458,
 4614,
 4964,
 5035,
 5192,
 5201,
 5243,
 5304,
 5715,
 6117,
 6377,
 6617,
 6754,
 6995,
 7143,
 7241,
 7500,
 7598,
 8173,
 8297,
 8513,
 9121,
 9164,
 9369,
 9396,
 9692,
 9804,
 10042,
 10351,
 10395,
 10518,
 10582,
 10807,
 10827,
 11029,
 11256,
 11285,
 11324,
 11343,
 11620,
 11640,
 11642,
 11712,
 11741,
 11837,
 12123,
 12197,
 12222,
 12349,
 12360,
 12705,
 13013,
 13319,
 13418,
 13507,
 13511,
 13647,
 13868,
 14147,
 14170,
 14427,
 14577,
 14687,
 14896,
 14923,
 14926,
 15419,
 15689,
 15897,
 15908,
 16128,
 16209,
 16322,
 16374,
 16393,
 16403,
 16504,
 16844,
 16908,
 16979,
 17087,
 17443,
 17507,
 18363,
 18683,
 19556,
 19585,
 19660,
 19764,
 20002,
 20275,
 20317,
 20320,
 20687,
 20741,
 20878,
 21014,
 21226,
 212

In [ ]:
# drop unroutable roads
filtered_df = df.drop(exception_cases_match_df).reset_index()

In [ ]:
# add column with road distances
filtered_df["Road distances [km]"] = route_distances

In [ ]:
# write to csv
filtered_df.to_csv('road_delivery_data.csv')